In [1]:
import pandas as pd

In [3]:
df=pd.read_csv("secondary_data_shuffled.csv", sep =";")
df

,class,cap-diameter,cap-shape,cap-surface,cap-color,does-bruise-or-bleed,gill-attachment,gill-spacing,gill-color,stem-height,...,stem-root,stem-surface,stem-color,veil-type,veil-color,has-ring,ring-type,spore-print-color,habitat,season
0,e,1.72,x,s,y,f,d,NaN,w,7.01,...,NaN,t,y,NaN,NaN,f,f,NaN,h,u
1,e,9.59,f,e,b,f,NaN,c,b,4.73,...,NaN,NaN,w,NaN,NaN,t,f,NaN,d,a
2,p,0.86,x,g,p,f,a,NaN,p,4.25,...,NaN,s,k,NaN,NaN,f,f,NaN,d,s
3,p,4.32,x,NaN,e,f,x,NaN,w,4.91,...,NaN,NaN,w,NaN,NaN,f,f,NaN,d,u
4,e,2.80,x,s,w,f,d,d,w,3.13,...,NaN,NaN,w,NaN,NaN,f,f,NaN,m,a
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61064,p,13.81,x,k,n,t,p,NaN,y,10.06,...,c,NaN,y,NaN,NaN,f,f,NaN,d,a
61065,p,2.69,x,d,w,f,d,d,p,3.76,...,NaN,h,w,NaN,NaN,f,f,NaN,g,a
61066,e,11.70,f,NaN,g,f,s,c,u,7.36,...,NaN,NaN,b,NaN,NaN,f,f,NaN,l,a
61067,p,2.83,s,w,u,f,a,c,u,5.76,...,NaN,s,u,NaN,NaN,f,f,NaN,d,a


In [7]:
df["class"].value_counts()

class
p    33888
e    27181
Name: count, dtype: int64